In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight

In [8]:
# Read CSV files from 'data' directory
df = pd.read_csv('df.csv')

In [9]:
df

,Unnamed: 0,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,departure_delay,departure_delay_categorised,arrival_delay,...,HourlyDryBulbTemperature_y,HourlyDryBulbTemperature_y_categorised,HourlyPrecipitation_y,HourlyPrecipitation_y_categorised,HourlyStationPressure_y,HourlyStationPressure_y_categorised,HourlyVisibility_y,HourlyVisibility_y_categorised,HourlyWindSpeed_y,HourlyWindSpeed_y_categorised
0,0,AS,121,SEA,ANC,2019-07-01,215,0,0,-16,...,57.0,0,0.0,0,30.04,1,10.0,0,3.0,0
1,3,F9,402,LAX,DEN,2019-07-01,143,-1,0,-7,...,62.0,0,0.0,0,24.69,0,10.0,0,8.0,0
2,4,F9,790,PDX,DEN,2019-07-01,156,21,1,10,...,62.0,0,0.0,0,24.70,0,8.0,0,10.0,0
3,5,F9,662,SFO,DEN,2019-07-01,158,106,2,84,...,62.0,0,0.0,0,24.70,0,8.0,0,10.0,0
4,6,AS,135,ORD,ANC,2019-07-01,400,-9,0,-4,...,57.0,0,0.0,0,30.06,1,10.0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492838,5512898,DL,3526,ATL,MGM,2019-05-31,59,-1,0,-18,...,74.0,0,0.0,0,29.67,1,10.0,0,6.0,0
5492839,5512899,DL,508,ATL,BHM,2019-05-31,55,-3,0,-6,...,74.0,0,0.0,0,29.26,0,10.0,0,0.0,1
5492840,5512900,DL,1075,ATL,HSV,2019-05-31,57,31,1,18,...,71.0,0,0.0,0,29.20,1,10.0,0,3.0,0
5492841,5512901,DL,2436,ATL,BHM,2019-05-31,56,28,1,13,...,67.0,0,0.0,0,29.24,1,10.0,0,0.0,1


In [10]:
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# Filtering the relevant columns for the features and target
feature_columns_dep = [col for col in df.columns if 'categorised' in col and '_x' in col]
feature_columns_arr = [col for col in df.columns if 'categorised' in col and '_y' in col]
target_columns = ['departure_delay_categorised', 'arrival_delay_categorised']

# Splitting the dataset into features (X) and targets (y) based on the departure and arrival delay
X_dep = df[feature_columns_dep]
X_arr = df[feature_columns_arr]
y_departure = df[target_columns[0]]
y_arrival = df[target_columns[1]]

# Splitting the dataset into training and test sets for departure and arrival delays
X_train_dep, X_test_dep, y_train_dep, y_test_dep = train_test_split(X_dep, y_departure, test_size=0.2, random_state=42)
X_train_arr, X_test_arr, y_train_arr, y_test_arr = train_test_split(X_arr, y_arrival, test_size=0.2, random_state=42)

# Since the classes are imbalanced, we will compute class weights for balancing
class_weights_dep = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_dep), y=y_train_dep)
class_weights_arr = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_arr), y=y_train_arr)

# Convert class weights to a dictionary to pass to RandomForestClassifier
class_weights_dep_dict = {i : class_weights_dep[i] for i,label in enumerate(np.unique(y_train_dep))}
class_weights_arr_dict = {i : class_weights_arr[i] for i,label in enumerate(np.unique(y_train_arr))}

# Re-initialising the Random Forest Classifier for departure and arrival delays with class weights
rf_classifier_dep_balanced = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights_dep_dict)
rf_classifier_arr_balanced = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights_arr_dict)

# Fitting the model on the training data for departure and arrival delays
rf_classifier_dep_balanced.fit(X_train_dep, y_train_dep)
rf_classifier_arr_balanced.fit(X_train_arr, y_train_arr)

# Making predictions on the test data for departure and arrival delays
y_pred_dep_balanced = rf_classifier_dep_balanced.predict(X_test_dep)
y_pred_arr_balanced = rf_classifier_arr_balanced.predict(X_test_arr)

# Evaluating the model's performance for departure and arrival delays with balanced classes
dep_accuracy_balanced = accuracy_score(y_test_dep, y_pred_dep_balanced)
dep_class_report_balanced = classification_report(y_test_dep, y_pred_dep_balanced)

arr_accuracy_balanced = accuracy_score(y_test_arr, y_pred_arr_balanced)
arr_class_report_balanced = classification_report(y_test_arr, y_pred_arr_balanced)

print(dep_accuracy_balanced)
print(dep_class_report_balanced)
print(arr_accuracy_balanced)
print(arr_class_report_balanced)


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# List of irrelevant columns to drop that don't contribute to the model
irrelevant_columns = ['Unnamed: 0', 'Unnamed: 0.1', 'scheduled_departure_dt', 
                      'scheduled_arrival_dt', 'actual_departure_dt', 'actual_arrival_dt', 
                      'cancelled_code', 'date', 'flight_number']

# Define the target variable
target_column = 'arrival_delay'

# Drop irrelevant columns from features
X = df.drop(columns=[target_column] + irrelevant_columns, errors='ignore')
y = df[target_column]

# Define numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Define transformers for numeric and categorical data preprocessing
numeric_transformer = SimpleImputer(strategy='median')
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create and configure the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate and print the evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)